# David Attenborough Speech Generator

Following notebook serves as means to train and test tacotron model producing David Attenborough speech.

First, upload the speech data to your Google Drive/DavidAttenboroughData/. Folder structure of the uploaded data is not really important as long as you also upload train, validation and test .txt files consisting of filenames and transcriptions. For instructions on how these files should be formatted, take a look [here](https://github.com/NVIDIA/tacotron2/tree/master/filelists).

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')
dataset_filepath = 'C:\\Users\\Jonathan\\Documents\\Other shit\\DavidAttenboroughSpeechGen\\data\\SirDavid'
project_root = '/content'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Initialize the environment

The following lines run Linux commands necessary to clone tacotron2 repository and install dependencies.

Afterwards sort out the imports


In [0]:
!git clone https://github.com/cpuimage/Tacotron-2.git
%cd '/content/Tacotron-2'
!pip install falcon inflect audioread Unidecode librosa matplotlib numpy scipy tqdm keras msgpack msgpack_numpy tensorflow_gpu==1.15
!apt install -y libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg libav-tools
%cd '..'

Cloning into 'Tacotron-2'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 43 (delta 6), reused 35 (delta 1), pack-reused 0
Unpacking objects: 100% (43/43), done.
/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Package 'libav-tools' has no installation candidate
/content


In [0]:
import os
os.chdir(os.path.join(ipynb_root, 'Tacotron-2'))
import argparse

import tensorflow as tf

import infolog
from hparams import hparams
from preprocess import write_metadata
from datasets.preprocessor import _process_utterance
from train import train, prepare_run

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





## Handle preprocessing

First, correct the wavs paths in .txt target files.


In [0]:
# this function is unique to current file structure. it probably won't work in general.
def change_wav_paths(txt_filepath):
  f = open(txt_filepath, "r")
  lines = f.read().split("\n")
  f.close()
  new_file = ""
  for line in lines:
    if len(line) > 0:
      line = line.split('|')
      transcript = line[1]
      # ... because of the next line.
      audio_filename = line[0].split('\\')[-2:]
      audio_filepath = os.path.join(dataset_filepath, *audio_filename)
      new_file += audio_filepath + '|' + transcript + "\n"
  f = open(txt_filepath, 'w')
  f.write(new_file)
  f.close()

In [0]:
change_wav_paths(os.path.join(dataset_filepath, 'sir_david_train.txt'))
change_wav_paths(os.path.join(dataset_filepath, 'sir_david_val.txt'))
change_wav_paths(os.path.join(dataset_filepath, 'sir_david_test.txt'))

Get your dataset and modify datasets/preprocessor.py -> "build_from_path" function.

In [0]:
def build_from_path(hparams, input_dir, output_dir):
  if isinstance(input_dir, str):
    input_dir = [input_dir]

  futures = []
  i = 1
  for _input_dir in input_dir:
    f = open(_input_dir, 'r')
    lines = f.readlines()
    f.close()

    for line in lines:
      wav_path, seq_text = line.strip().split('|')
      futures.append(_process_utterance(output_dir, i, wav_path, seq_text, hparams))
      i += 1

  return futures

In [0]:
input_dir = [os.path.join(dataset_filepath, txt) for txt in ['sir_david_train.txt', 'sir_david_val.txt', 'sir_david_test.txt']]
output_dir = os.path.join(ipynb_root, "data")
os.makedirs(output_dir, exist_ok=True)

metadata = build_from_path(hparams, input_dir, output_dir)
write_metadata(metadata, output_dir)

Write 4624 utterances, 1306762.0 mel frames, 261352400.0 audio timesteps, (4.54 hours)
Max input length (text chars): 83
Max mel frames length: 585
Max audio timesteps length: 117000


## Training

Train your Tacotron model. Yields the logs-Tacotron2 folder.

In [0]:
def main(argv):
  parser = argparse.ArgumentParser()
  # these arguments prevent jupyter notebook from throwing an error.
  parser.add_argument('-f')
  # these arguments are actually used for training. Change the default parameter if you want to change them.
  parser.add_argument('--base_dir', default=ipynb_root)
  parser.add_argument('--hparams', default='', help='Hyperparameter overrides as a comma-separated list of name=value pairs')
  parser.add_argument('--train_input', default=os.path.join(ipynb_root, 'data/train.txt'))
  parser.add_argument('--name', help='Name of logging directory.')
  parser.add_argument('--model', default='SirDavid_first_run')
  parser.add_argument('--output_dir', default=os.path.join(ipynb_root, 'output'), help='folder to contain synthesized mel spectrograms')
  parser.add_argument('--restore', type=bool, default=True, help='Set this to False to do a fresh training')
  parser.add_argument('--summary_interval', type=int, default=250, help='Steps between running summary ops')
  parser.add_argument('--embedding_interval', type=int, default=5000, help='Steps between updating embeddings projection visualization')
  parser.add_argument('--checkpoint_interval', type=int, default=2500, help='Steps between writing checkpoints')
  parser.add_argument('--eval_interval', type=int, default=5000, help='Steps between eval on test data')
  parser.add_argument('--train_steps', type=int, default=100000, help='total number of tacotron2 training steps')
  parser.add_argument('--tf_log_level', type=int, default=3, help='Tensorflow C++ log level.')
  parser.add_argument('--slack_url', default=None, help='slack webhook notification destination link')
  parser.add_argument('--gpu_devices', default='0', help='Set the gpu the model should run on.(eg:hvd,0,1,2...)')
  # args = parser.parse_args(argv[1:])
  args = parser.parse_args()
  use_hvd = args.gpu_devices == 'hvd'
  if not use_hvd:
      os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_devices
  log_dir, _hparams = prepare_run(args)
  train(log_dir, args, _hparams, use_hvd=use_hvd)

In [0]:
# run training
tf.app.run(main=main)

Checkpoint path: /content/logs-SirDavid_first_run/pretrained/model.ckpt
Loading training data from: /content/data/train.txt
Using model: SirDavid_first_run
Hyperparameters:
  adam_beta1: 0.9
  adam_beta2: 0.999
  adam_epsilon: 1e-06
  allow_clipping_in_normalization: True
  attention_dim: 128
  batch_size: 32
  clip_gradients: True
  clip_mels_length: True
  clip_outputs: True
  cross_entropy_pos_weight: 20
  data_random_state: 1234
  decay_learning_rate: True
  decay_rate: 0.5
  decay_steps: 50000
  decoder_layers: 2
  decoder_lstm_units: 512
  dropout_rate: 0.5
  embedding_dim: 256
  enc_conv_channels: 128
  enc_conv_kernel_size: (5,)
  enc_conv_num_layers: 3
  encoder_lstm_units: 128
  final_learning_rate: 1e-05
  fine_tuning: False
  frame_shift_ms: None
  griffin_lim_iters: 60
  hop_size: 200
  initial_learning_rate: 0.001
  lower_bound_decay: 0.1
  magnitude_power: 2.0
  mask_decoder: False
  mask_encoder: True
  max_abs_value: 4.0
  max_iters: 2000
  max_mel_frames: 1000
  min_l

W0329 21:44:44.606408 139754210260864 module_wrapper.py:139] From /content/Tacotron-2/train.py:152: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



W0329 21:44:44.608798 139754210260864 module_wrapper.py:139] From /content/Tacotron-2/train.py:156: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Loaded metadata for 4624 examples (4.54 hours)



W0329 21:44:44.644646 139754210260864 module_wrapper.py:139] From /content/Tacotron-2/feeder.py:80: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



...

Saving eval log to /content/logs-SirDavid_first_run/eval-dir..
Eval loss for global step 20000: 2.737
Writing eval summary!

Saving alignment, Mel-Spectrograms and griffin-lim inverted waveform..
Input at step 20000: ^it is itself an active volcano.~______________________

Saving Model Character Embeddings visualization..
Tacotron2 Character embeddings have been updated on tensorboard!

Generated 32 train batches of size 32 in 0.859 sec

Generated 32 train batches of size 32 in 1.052 sec

Generated 32 train batches of size 32 in 1.068 sec

Generated 32 train batches of size 32 in 1.172 sec

Generated 32 train batches of size 32 in 1.018 sec

Generated 32 train batches of size 32 in 0.964 sec

Generated 32 train batches of size 32 in 0.969 sec
Step   20250 [1.122 sec/step, loss=2.31852, avg_loss=2.14315]
Writing summary at step 20250

Generated 32 train batches of size 32 in 1.023 sec

Generated 32 train batches of size 32 in 1.027 sec

Generated 32 train batches of size 32 in 0.982 sec

  0%|          | 0/7 [00:00<?, ?it/s]


Running evaluation at step 25000


100%|██████████| 7/7 [00:12<00:00,  1.78s/it]


Saving eval log to /content/logs-SirDavid_first_run/eval-dir..
Eval loss for global step 25000: 2.829
Writing eval summary!

Saving alignment, Mel-Spectrograms and griffin-lim inverted waveform..
Input at step 25000: ^there are more than 7,000 individuals on fernandina alone.~______________________

Saving Model Character Embeddings visualization..
Tacotron2 Character embeddings have been updated on tensorboard!

Generated 32 train batches of size 32 in 0.908 sec

Generated 32 train batches of size 32 in 1.040 sec

Generated 32 train batches of size 32 in 1.139 sec

Generated 32 train batches of size 32 in 0.997 sec

Generated 32 train batches of size 32 in 1.075 sec

Generated 32 train batches of size 32 in 1.103 sec

Generated 32 train batches of size 32 in 0.926 sec

Generated 32 train batches of size 32 in 1.110 sec
Step   25250 [1.156 sec/step, loss=2.22331, avg_loss=2.09119]
Writing summary at step 25250

Generated 32 train batches of size 32 in 1.071 sec

Generated 32 train batc

## Evaluate

Synthesize/Evaluate the Tacotron model. Gives the synth_output folder.